In [34]:
from collections import defaultdict
import numpy as np
import operator
import os
import pandas as pd
import pickle

import logging
logging.basicConfig(level=logging.DEBUG,
        format='%(levelname)-8s [%(lineno)d] %(message)s')

In [35]:
weight = 'salience'
rank = 2

In [36]:
_, index = pickle.load(open(
    '/mnt/store/home/makrai/project/verb-tensor/depCC/sparstensr_{}_2.pkl'.format(weight), mode='rb'))

# [SimVerb](http://people.ds.cam.ac.uk/dsg40/simverb.html) (Gerz+ EMNLP 2016)

In [37]:
simverb = pd.read_csv('/mnt/permanent/Language/English/Data/verb-similarity/simverb-3500/SimVerb-3500.txt', sep='\t',
                      header=None, names=['verb1', 'verb2', 'pos', 'sim', 'rel'])

In [38]:
simverb.head()

,verb1,verb2,pos,sim,rel
0,take,remove,V,6.81,SYNONYMS
1,walk,trail,V,4.81,COHYPONYMS
2,feed,starve,V,1.49,ANTONYMS
3,shine,polish,V,7.80,SYNONYMS
4,calculate,add,V,5.98,HYPER/HYPONYMS


In [39]:
simverb.shape

(3500, 5)

In [40]:
simverb.groupby('rel').sim.describe(percentiles=[]).sort_values('count', ascending=False)

,count,mean,std,min,50%,max
rel,,,,,,
NONE,2093.0,3.431276,2.342695,0.0,3.150,9.79
HYPER/HYPONYMS,800.0,6.012525,2.104537,0.5,6.310,9.96
SYNONYMS,306.0,6.789150,2.104490,0.5,7.140,9.96
COHYPONYMS,190.0,4.435526,2.381992,0.0,4.665,9.30
ANTONYMS,111.0,0.977748,1.074232,0.0,0.660,6.04


## Tensor

In [41]:
_, index = pickle.load(open(
    '/mnt/store/home/makrai/project/verb-tensor/depCC/sparstensr_{}_2.pkl'.format(weight), mode='rb'))

In [42]:
oov = defaultdict(int)

def tensor_based_sim(rank):
    ktensor, fit, n_iterations, exectimes = pickle.load(open(
        '/mnt/store/home/makrai/project/verb-tensor/depCC/ktensor_{}_2_{}.pkl'.format(weight, rank), mode='rb'))
    #modes = ['nsubj', 'ROOT', 'dobj']
    ktensor.U[1] /= np.apply_along_axis(np.linalg.norm, 1, ktensor.U[1]).reshape((-1,1))
    def pointwise_prod_more(ser):
        try:
            #return ktensor.lmbda .dot( 
            return ktensor.U[1][index['ROOT'][ser[0]]] .dot( ktensor.U[1][index['ROOT'][ser[1]]])
        except KeyError as e:
            oov[e.args] += 1
            return np.nan
    simverb['tensor_sim_{}'.format(rank)] = simverb[['verb1', 'verb2']].apply(pointwise_prod_more, axis=1)

In [43]:
sorted(oov.items(), key=operator.itemgetter(1), reverse=True)

[]

In [44]:
for exp in range(1, int(np.log2(rank))+1):
    tensor_based_sim(rank=2**exp)

/home/makrai/tool/python/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/home/makrai/tool/python/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


In [45]:
simverb.corr()

,sim,tensor_sim_2
sim,1.000000,0.005952
tensor_sim_2,0.005952,1.000000


In [46]:
simverb.groupby('rel').corr().sim

rel                         
ANTONYMS        sim             1.000000
                tensor_sim_2    0.006088
COHYPONYMS      sim             1.000000
                tensor_sim_2    0.044773
HYPER/HYPONYMS  sim             1.000000
                tensor_sim_2    0.049772
NONE            sim             1.000000
                tensor_sim_2   -0.013824
SYNONYMS        sim             1.000000
                tensor_sim_2   -0.039185
Name: sim, dtype: float64

# [Transitive sentence similarity](http://www.cs.ox.ac.uk/activities/compdistmeaning/GS2011data.txt) (Grefenstette and Sadrzadeh, EMNLP 2011)

In [47]:
svo_sim = pd.read_csv('/mnt/permanent/Language/English/Data/verb-similarity/GS2011data.txt', sep=' ', 
                 true_values=['HIGH'], false_values=['LOW'])

In [48]:
cols_ordered = ["subject", "verb", "landmark", "object", "input", "hilo", "participant"]

In [49]:
svo_sim[cols_ordered].head()

,subject,verb,landmark,object,input,hilo,participant
0,family,provide,supply,home,4,True,participant20
1,government,provide,leave,cash,3,False,participant20
2,government,provide,supply,cash,7,True,participant20
3,man,provide,leave,money,6,True,participant20
4,man,provide,supply,money,7,True,participant20


In [50]:
svo_sim.describe(percentiles=[])

,input
count,2500.000000
mean,3.570800
std,2.193561
min,0.000000
50%,3.000000
max,7.000000


In [51]:
svo_sim.corr(method='spearman')

,input,hilo
input,1.00000,0.51689
hilo,0.51689,1.00000


## Tensor

In [52]:
%time ktensor, fit, n_iterations, exectimes = pickle.load(open('/mnt/store/home/makrai/project/verb-tensor/depCC/ktensor_{}_2_{}.pkl'.format(weight, rank), mode='rb'))
#modes = ['nsubj', 'ROOT', 'dobj']

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 3.01 ms


In [53]:
#ktensor.U[1] /= np.apply_along_axis(np.linalg.norm, 1, ktensor.U[1]).reshape((-1,1))

In [54]:
oov = defaultdict(int)
def verb_pred(ser, prec_at=10):
    try:
        predicted_ids = np.argsort((-ktensor.lmbda * 
                                    ktensor.U[0][index['nsubj'][ser[0]]]) .dot(
            (ktensor.U[1] * ktensor.U[2][index['dobj'][ser[0]]]).T))#] # 
        return [index['ROOT'].inverse[i] for i in predicted_ids[:prec_at]]
    except KeyError as e:
        oov[e.args] += 1
        return []

In [55]:
%time svo_sim['predicted'] = svo_sim[['subject', 'object']].apply(verb_pred, axis=1)

CPU times: user 36.9 s, sys: 1min 52s, total: 2min 29s
Wall time: 11.2 s


In [56]:
sorted(oov.items(), key=operator.itemgetter(1), reverse=True)

[(('tribunal',), 26),
 (('runway',), 26),
 (('cinema',), 24),
 (('spokesman',), 24)]

In [57]:
def is_good(ser):
    #return ser['verb'] in ser['predicted']# or 
    return ser['landmark'] in ser['predicted']
        
svo_sim['good'] = svo_sim.apply(is_good, axis=1)

In [58]:
svo_sim.good.sum()

24

In [59]:
svo_sim.shape

(2500, 9)

|verb|`* 1`|`* lmbda`|
|----|--|------|
|unnorm|130|**272**|
|norm|0|24|

Majoroty baseline...

In [60]:
cols_ordered = ["subject", "verb", "landmark", "object", "input", "hilo", "participant", 'predicted', 'good']

In [61]:
svo_sim[cols_ordered].sample()

,subject,verb,landmark,object,input,hilo,participant,predicted,good
1464,sight,meet,satisfy,eye,3,False,participant17,"[comentou, ......................, heartbreak,...",False


# [Verb prediction (tensor)](https://everest.hds.utc.fr/doku.php?id=en:lfmnips12) (Jenatton+ NIPS 2012)

In [62]:
dataset_dir = '/mnt/permanent/Language/English/Data/verb-similarity/SVO-tensor-dataset/'
def get_index(pos):
    with open(os.path.join(dataset_dir, 'svo-{}s.lst'.format(pos))) as infile:
        return {i+1: ' '.join(line.strip().split('_')[2:-1]) for i, line in enumerate(infile)}

In [63]:
svo_df = pd.read_csv(os.path.join(dataset_dir, 'svo_data_train_1000000.dat'), sep='\t', header=None, 
                     names=['subject', 'verb', 'object'])

In [64]:
index_verb = get_index('verb')
index_noun = get_index('noun')

In [65]:
svo_df.subject = svo_df.subject.apply(index_noun.get)
svo_df.verb = svo_df.verb.apply(index_verb.get)
svo_df.object = svo_df.object.apply(index_noun.get)

In [66]:
svo_df.head(10)

,subject,verb,object
0,man,swipe,credit card
1,george westinghouse,illuminate,exposition
2,personality,vanish,moment
3,fable,highlight,role
4,secretion,call,surfactant
5,device,store,multimedia
6,diversity,base,engineering
7,troop,destroy,weather station
8,flute,include,vein
9,complication,relate,emphysema
